In [70]:
import os, json

from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.sequential import SequentialChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
    )
from config import LOGGER

from langchain.chains.qa_with_sources.retrieval import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI


In [80]:
MAIN_DIR = ".."
ARTIFACT_DIR = os.path.join(MAIN_DIR, "artifacts")
EMB_DIR = os.path.join(MAIN_DIR, "data", "emb_store", "uc", "faiss", "text-embedding-ada-002")

with open(os.path.join(MAIN_DIR, "auth", "api_keys.json"), "r") as f:
    api_keys = json.load(f)
    
os.environ["OPENAI_API_KEY"] = api_keys["OPENAI_API_KEY"]

with open(os.path.join(MAIN_DIR, "data", "queries", "uc_all.txt"), "r") as f:
    test_cases_txt = f.readlines()
    
with open(os.path.join(MAIN_DIR, "data", "queries", "uc_all_emb.json"), "r") as f:
    test_cases_emb = json.load(f)
    
test_cases = [(txt, emb) for txt, emb in zip(test_cases_txt, test_cases_emb)]
print("Number of test cases:", len(test_cases))
print("Length of embeddings:", len(test_cases[0][1]))

Number of test cases: 30
Length of embeddings: 1536


In [55]:
db_name = "v8-add-tables_2500_500"
embeddings = OpenAIEmbeddings()
docsearch = FAISS.load_local(os.path.join(EMB_DIR, db_name), embeddings)

In [88]:
# CHAT PROMTP TEMPLATE
system_prompt = """
You are a physician assistant giving advice on treatment for moderate to severe ulcerative colitis (UC).
Make reference to the CONTEXT given to assess the scenario.
If the answer cannot be inferred from CONTEXT, return "NO ANSWER", don't try to make up an answer.
==========
TASK: ANALYSE the given patient profile based on given query based on one of the following criteria:
- Whether treated patient is new patient or patient under maintenance
- Prior response to Infliximab
- Prior failure to Anti-TNF agents
- Prior failure to Vedolizumab
- Age
- Pregnancy
- Extraintestinale manifestations
- Pouchitis

FINALLY RETURN up to 2 TOP choices of biological drugs given patient profile and context. Explain the PROS and CONS of the 2 choices.
If answer cannot be derived from context, RETURN "NO ANSWER" and explain reason.
==========
OUTPUT INSTRUCTIONS:
Output your answer as a list of JSON objects with keys: drug_name, advantages, disadvantages.
==========
CONTEXT:
{summaries}
==========
"""

QA_PROMPT = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            system_prompt, input_variables=["summaries"]
        ),
        HumanMessagePromptTemplate.from_template("PATIENT PROFILE: {question}"),
    ]
)

In [96]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=512)
# llm = ChatOpenAI(model_name="gpt-4", temperature=0, max_tokens=512)
qa_with_sources_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_PROMPT},
    max_tokens_limit=3375, reduce_k_below_max_tokens=True,
    verbose=True
    )

In [97]:
sample_testcase = test_cases_txt[11]
sample_answer = qa_with_sources_chain(sample_testcase)
sample_answer



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.


{'question': '52 year-old woman with moderate to severe distal ulcerative colitis that had a successful induction with vedolizumab. What would be the maintenance therapy?\n',
 'answer': 'Based on the given patient profile, the patient is a 52-year-old woman with moderate to severe distal ulcerative colitis who had a successful induction with vedolizumab. The query is about the appropriate maintenance therapy for this patient.\n\nBased on the information provided, the patient has responded well to vedolizumab during the induction phase. Therefore, vedolizumab can be considered as a suitable maintenance therapy for this patient. Vedolizumab is an advanced therapy that specifically targets the gut-homing lymphocytes, reducing inflammation in the gut. It has been shown to be effective in maintaining remission in patients with ulcerative colitis.\n\nAnother option for maintenance therapy could be adalimumab. Adalimumab is another advanced therapy that has been approved for the treatment of 

In [107]:
refine_prompt = """You are a senior physician giving advice on treatment for moderate to severe ulcerative colitis (UC).
Make reference to the CONTEXT given to assess the scenario.
==========
TASK: Your task is to analyse critically the CURRENT ANSWER given the CONTEXT.
Then refine the existing answer (only if needed) based on the following instructions:
- If no refinement needed, return the original answer in the correct output format.
- If the answer cannot be inferred from CONTEXT, return "NO ANSWER".
- The final output should be a list of up to 2 choices of biological drugs given patient profile and context and PROS/CONS for each recommendations.
==========
CONTEXT:
{source_documents}
==========
ORIGINAL QUESTION: ANALYSE the given patient profile based on given query based on one of the following criteria:
- Whether treated patient is new patient or patient under maintenance
- Prior response to Infliximab
- Prior failure to Anti-TNF agents
- Prior failure to Vedolizumab
- Age
- Pregnancy
- Extraintestinale manifestations
- Pouchitis
RETURN up to 2 TOP choices of biological drugs given patient profile and context. Explain the PROS and CONS of the 2 choices.
PATIENT PROFILE: {question}
==========
CURRENT ANSWER:
{answer}
==========
REFINED ANSWER:
"""

REFINE_PROMPT = PromptTemplate(
    template=refine_prompt, input_variables = ["source_documents", "question", "answer"]
)

In [71]:
llm_chain = LLMChain(
    llm=llm,
    prompt=REFINE_PROMPT,
    verbose=True,
    output_key="text"
)

refine_chain = StuffDocumentsChain(
    llm_chain = llm_chain,
    input_key="source_documents",
    output_key="final_answer",
    document_variable_name = "source_documents",
    document_separator="\n----------\n"
)

In [ ]:
sample_refined_answer = refine_chain(
    inputs={
        "question": sample_answer["question"],
        "answer": sample_answer["answer"],
        "source_documents": sample_answer["source_documents"]
    }
)

sample_refined_answer

In [73]:
qa_chain_with_refine = SequentialChain(
    chains = [qa_with_sources_chain, refine_chain],
    input_variables = ["question"],
    output_variables = ["answer", "source_documents", "final_answer"],
    verbose = True
)

In [ ]:
qa_chain_with_refine(sample_testcase)

In [117]:
from exp import QuestionAnsweringWithIndexSearchExperiment
from typing import Union, Optional, Literal, List
from custom_parsers import DrugOutput
from langchain.output_parsers import PydanticOutputParser
import re
import pandas as pd
from datetime import datetime

class QAWithIndexSearchandRefineExperiment(QuestionAnsweringWithIndexSearchExperiment):
    def __init__(
        self,
        original_prompt: Union[PromptTemplate, ChatPromptTemplate],
        refine_prompt: PromptTemplate,
        vector_store: str,
        llm_type: str = "gpt-3.5-turbo",
        refine_llm_type: Optional[str] = None,
        emb: str = "text-embedding-ada-002",
        keys_json: str = os.path.join(MAIN_DIR, "auth", "api_keys.json"),
        temperature: float = 0,
        max_tokens: int = 512,
        gt: Optional[str] = None,
        verbose: bool = False,
        k: int = 4,
        max_tokens_limit: int = 3375,
        reduce_k_below_max_tokens: bool = True,
    ):
        self.temperature = temperature
        self.max_tokens = max_tokens
        
        with open(keys_json, "r") as f:
            self.keys = json.load(f)

        self.openai_key = self.keys["OPENAI_API_KEY"]
        self.ground_truth = self.load_groundtruth(gt) if gt else None
        self.chain = None
        self.verbose = verbose

        self.k = k
        self.max_tokens_limit = max_tokens_limit
        self.reduce_k_below_max_tokens = reduce_k_below_max_tokens
        self.questions = []
        self.original_answers = []
        self.final_answers = []
        self.sources = []
        self.drug_parser = PydanticOutputParser(pydantic_object=DrugOutput)
        
        self.original_prompt = original_prompt
        self.refine_prompt = refine_prompt
        
        self.answer_llm = ChatOpenAI(
            model_name = llm_type, temperature = temperature, max_tokens = max_tokens,
            openai_api_key=self.openai_key
        )
        
        self.refine_llm = ChatOpenAI(
            model_name=refine_llm_type, temperature=temperature, max_tokens=max_tokens,
            openai_api_key=self.openai_key
        ) if refine_llm_type else self.answer_llm
        
        print("Answer LLM:", self.answer_llm.model_name)
        print("Refine LLM:", self.refine_llm.model_name)
        
        self.embedder = OpenAIEmbeddings(model=emb, openai_api_key=self.openai_key)
        try:
            self.load_vectorstore(vector_store)
        except Exception:
            print(
                "Vectorstore invalid. Please load valid vectorstore or create new vectorstore."
            )
            
    def _create_chain(
        self,
        chain_type: Literal["stuff", "map_reduce", "map_rerank", "refine"] = "stuff",
        return_source_documents: bool = True,
    ):
        self.qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
            llm=self.answer_llm,
            chain_type=chain_type,
            retriever=self.docsearch.as_retriever(search_kwargs={"k": self.k}),
            return_source_documents=return_source_documents,
            chain_type_kwargs={"prompt": self.original_prompt},
            max_tokens_limit=self.max_tokens_limit,
            reduce_k_below_max_tokens=self.reduce_k_below_max_tokens,
            verbose=self.verbose
        )
        
        self.refine_chain = StuffDocumentsChain(
            llm_chain=LLMChain(llm=self.refine_llm, prompt=self.refine_prompt, verbose=self.verbose),
            input_key="source_documents",
            output_key="final_answer",
            document_variable_name = "source_documents",
            document_separator="\n----------\n"
        )
        
        self.chain = SequentialChain(
            chains = [self.qa_chain, self.refine_chain],
            input_variables = ["question"],
            output_variables = ["answer", "source_documents", "final_answer"],
            verbose = True
            )
        
    def reset(self):
        """Reset queries and answers"""
        self.questions = []
        self.original_answers = []
        self.final_answers = []
        self.sources = []
        
    def save_json(self, output_path: str):
        """Save Output of test case runs to json file

        Args:
            output_path (str): Output Path to json file.
        """
        output_dict = {}
        output_dict["original_prompt"] = QuestionAnsweringWithIndexSearchExperiment.convert_prompt_to_string(
            self.original_prompt
        )
        output_dict["refine_prompt"] = QuestionAnsweringWithIndexSearchExperiment.convert_prompt_to_string(
            self.refine_prompt
        )
        output_dict["test_cases"] = []

        for question, original_answer, final_answer, source in zip(self.questions, self.original_answers, self.final_answers, self.sources):
            output_dict["test_cases"].append(
                {"question": question, "original_answer": original_answer, "final_answer": final_answer, "sources": source}
            )

        with open(output_path, "w") as f:
            json.dump(output_dict, f)
            
    def load_json(self, json_path: str, reset: bool = False):
        """Load Queries and Answers from Json file

        Args:
            json_path (str): Path to json output file to load into instance
            reset (bool, optional): If reset, clear queries and answers from memory before loading. Defaults to False.
        """
        if reset:
            self.reset()
        with open(json_path, "r") as f:
            input_dict = json.load(f)
        for test_case in input_dict["test_cases"]:
            self.questions.append(test_case["question"])
            self.original_answers.append(test_case["original_answer"])
            self.final_answers.append(test_case["final_answer"])
            self.sources.append(test_case["sources"])
        LOGGER.info("Json file loaded successfully into Experiment instance.")
        
    def run_test_cases(
        self,
        test_cases: Union[List[str], str],
        only_return_source: bool = False,
        chain_type: Literal["stuff", "refine", "map_reduce", "map_rerank"] = "stuff",
    ):
        if isinstance(test_cases, str):
            with open(test_cases, "r", encoding="utf-8-sig") as f:
                test_cases = f.readlines()
            test_cases = [test_case.rstrip() for test_case in test_cases]

        if not self.chain:
            self._create_chain(chain_type=chain_type)

        if only_return_source:
            LOGGER.info("Perform Semantic Search for Source Documents only (No QA).")
            
        for test_case in test_cases:
            print("Query: {}".format(test_case))
            sources = []  # All sources for 1 single query
            if only_return_source:
                self.questions.append(test_case)
                self.original_answers.append(None)
                self.final_answers.append(None)
                inputs = {"question": test_case}
                source_documents = self.qa_chain._get_docs(inputs)

            else:
                output = self.chain(test_case)
                self.questions.append(output["question"])
                self.original_answers.append(output["answer"])
                self.final_answers.append(output["final_answer"])
                source_documents = output["source_documents"]

            token_count = 0
            for document in source_documents:
                sources.append(
                    {
                        "title": document.metadata["title"],
                        "filename": document.metadata["source"].split("/")[-1],
                        "page": document.metadata["page"],
                        "text": document.page_content,
                    }
                )
                token_count += (
                    self.qa_chain.combine_documents_chain.llm_chain.llm.get_num_tokens(
                        document.page_content
                    )
                )

            self.sources.append(sources)
            LOGGER.info(
                f"{token_count} tokens were added to prompt from {len(source_documents)} documents"
            )
            
    def write_csv(self, output_csv: str, num_docs: int = 10):
        """Write questions and answers to .csv files

        Args:
            output_csv (str): Path to output csv file
        """

        pd_answers = [[], []]
        pd_sources = [[] for _ in range(num_docs)]

        for answer, sources in zip(self.final_answers, self.sources):
            if answer:
                drugs_info = re.findall(re.compile(r"{[^{}]+}"), answer)
                drugs = []
                for drug in drugs_info:
                    try:
                        drug = self.drug_parser.parse(drug)
                        drugs.append(drug)
                    except Exception:
                        pass
            else:
                drugs = []

            pd_answers[0].append(drugs[0].drug_name if len(drugs) > 0 else None)
            pd_answers[1].append(drugs[1].drug_name if len(drugs) > 1 else None)

            for idx, source in enumerate(sources):
                pd_sources[idx].append(
                    QuestionAnsweringWithIndexSearchExperiment.process_source(source)
                )

            if idx + 1 < len(pd_sources):
                for i in range(idx + 1, len(pd_sources)):
                    pd_sources[i].append(None)

        info = {"question": self.questions}

        if self.ground_truth is not None:
            info["gt_rec1"] = self.ground_truth["Recommendation 1"].tolist()
            info["gt_rec2"] = self.ground_truth["Recommendation 2"].tolist()
            info["gt_rec3"] = self.ground_truth["Recommendation 3"].tolist()
            info["gt_avoid"] = self.ground_truth["Drug Avoid"].tolist()
            info["gt_reason"] = self.ground_truth["Reasoning"].tolist()

        info["original_prompt"] = [
            QuestionAnsweringWithIndexSearchExperiment.convert_prompt_to_string(self.original_prompt)
        ] * len(self.questions)

        info["refine_prompt"] = [
            QuestionAnsweringWithIndexSearchExperiment.convert_prompt_to_string(self.refine_prompt)
        ] * len(self.questions)
        
        info["raw_original_answer"] = self.original_answers
        info["raw_final_answer"] = self.final_answers
        info["answer1"] = pd_answers[0]
        info["answer2"] = pd_answers[1]

        for idx, pd_source in enumerate(pd_sources):
            info[f"source{idx+1}"] = pd_source

        panda_df = pd.DataFrame(info)

        panda_df.to_csv(output_csv, header=True)

In [120]:
exp_args = {
    "original_prompt": QA_PROMPT,
    "refine_prompt": REFINE_PROMPT,
    "vector_store": os.path.join(MAIN_DIR, "data", "emb_store", "uc", "faiss", "text-embedding-ada-002", "v8-add-tables_2500_500"),
    "llm_type": "gpt-4",
    "refine_llm_type": "gpt-4",
    "temperature": 0,
    "max_tokens": 512,
    "gt": os.path.join(MAIN_DIR, "data", "queries", "uc_all_gt.csv"),
    "verbose": True,
    "k": 10,
    "max_tokens_limit": 6500,
    "reduce_k_below_max_tokens": True
}

refine_exp = QAWithIndexSearchandRefineExperiment(
    **exp_args
)

refine_exp.reset()
refine_exp.run_test_cases(
    test_cases_txt,
    chain_type="stuff"
)

description = "ChatGround_Refine_Tables_Chunk-size=2500_Overlap=500_Doc={}_Max-token={}_Stuff".format(exp_args["k"], exp_args["max_tokens_limit"])
save_path = os.path.join(
    ARTIFACT_DIR,
    "{}_{}_{}".format(
        exp_args["llm_type"], description, datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    ),
)
os.makedirs(save_path, exist_ok=True)
refine_exp.save_json(
    os.path.join(save_path, f"result.json")
)
refine_exp.write_csv(
    os.path.join(save_path, f"result.csv"), num_docs=exp_args["k"]
)

Answer LLM: gpt-4
Refine LLM: gpt-4
2023-10-02 17:12:29,356:INFO: Successfully loaded existing vectorstore from local storage
Query: 40 year old male with newly diagnosed moderate UC and articular extraintestinal manifestations



> Entering new SequentialChain chain...


> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are a senior physician giving advice on treatment for moderate to severe ulcerative colitis (UC).
Make reference to the CONTEXT given to assess the scenario.
TASK: Your task is to analyse critically the CURRENT ANSWER given the CONTEXT.
Then refine the existing answer (only if needed) based on the following instructions:
- If no refinement needed, return the original answer in the correct output format.
- If the answer cannot be inferred from CONTEXT, return "NO ANSWER".
- The final output should be a list of up to 2 choices of biological drugs given patient profile and context and PRO

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens p


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are a senior physician giving advice on treatment for moderate to severe ulcerative colitis (UC).
Make reference to the CONTEXT given to assess the scenario.
TASK: Your task is to analyse critically the CURRENT ANSWER given the CONTEXT.
Then refine the existing answer (only if needed) based on the following instructions:
- If no refinement needed, return the original answer in the correct output format.
- If the answer cannot be inferred from CONTEXT, return "NO ANSWER".
- The final output should be a list of up to 2 choices of biological drugs given patient profile and context and PROS/CONS for each recommendations.
CONTEXT:
prognosis, with 1 study reporting fewer hospitalizations and
less frequent requirement of immunosuppression in pa-
tients diagnosed after age 60 years,47 and a meta-analysis
suggested
that,
although
elderly
IBD
presents
less
commonly with complications, there are similar or higher
r

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens p


> Finished chain.

> Finished chain.
2023-10-02 17:14:37,403:INFO: 5593 tokens were added to prompt from 10 documents
Query: 35 year old male with known moderate UC with prior exposure to infliximab but has worsening colitis on endoscopy despite compliance



> Entering new SequentialChain chain...


> Entering new RetrievalQAWithSourcesChain chain...


...


> Finished chain.

> Finished chain.
2023-10-02 17:48:12,116:INFO: 5544 tokens were added to prompt from 10 documents
Query: 38 year-old woman with moderate to severe ulcerative colitis and pulmonary tuberculosis which was treated 2 years ago.


> Entering new SequentialChain chain...


> Entering new RetrievalQAWithSourcesChain chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are a senior physician giving advice on treatment for moderate to severe ulcerative colitis (UC).
Make reference to the CONTEXT given to assess the scenario.
TASK: Your task is to analyse critically the CURRENT ANSWER given the CONTEXT.
Then refine the existing answer (only if needed) based on the following instructions:
- If no refinement needed, return the original answer in the correct output format.
- If the answer cannot be inferred from CONTEXT, return "NO ANSWER".
- The final output should be a list of up to 2 choices of biological drugs given patient profile and context and PROS/CONS for each recommendations.
CONTEXT:
Burr et al. 
 
7 of 45 
INTRODUCTION 
  Ulcerative colitis (UC) is a chronic inflammatory disorder of the bowel that causes 
continuous mucosal inflammation commencing in the rectum and extending proximally for a 
variable extent.[1] It is estimated that UC affects 2.5 million peopl

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for 10KTPM-200RPM in organization org-ETbTMTN2JXYOUPDJgxuH2iO6 on tokens p


> Finished chain.

> Finished chain.
2023-10-02 17:49:27,063:INFO: 6061 tokens were added to prompt from 10 documents
